In [ ]:
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from openpilot.selfdrive.car.ford.values import CAR

database = get_comma_car_segments_database()

platforms = [c.value for c in CAR]

In [2]:
from openpilot.tools.lib.logreader import get_first_message, LogReader


FW_TO_CHECK = set()

for platform in platforms:
  if platform not in database:
    print(f"Skipping platform: {platform}, no data available")
    continue

  for segment in database[platform]:
    lr = LogReader(segment)
    CP = get_first_message(lr, "carParams").carParams
    FW_TO_CHECK.add((CP.carVin, CP.carFw, CP.carFingerprint, CP.fuzzyFingerprint, CP.fingerprintSource))

Skipping platform: FORD F-150 14TH GEN, no data available


In [6]:
from openpilot.selfdrive.car.fw_versions import build_fw_dict
from openpilot.selfdrive.car.ford.values import match_fw_to_car_fuzzy
from openpilot.selfdrive.car.ford.fingerprints import FW_VERSIONS

success, count = 0, 0
for vin, car_fw, expected_platform, fuzzy, surce in FW_TO_CHECK:
  if len(car_fw) == 0:
    print(f"Skipping vin: {vin} platform: {expected_platform}, no car fw")
    continue
  car_fw = build_fw_dict(car_fw)
  fuzzy_fingerprint = match_fw_to_car_fuzzy(car_fw, FW_VERSIONS)
  if len(fuzzy_fingerprint) == 0:
    fuzzy_fingerprint = "mock"
  elif len(fuzzy_fingerprint) > 1:
    fuzzy_fingerprint = "multiple"
  else:
    fuzzy_fingerprint = list(fuzzy_fingerprint)[0]

  correct = fuzzy_fingerprint == expected_platform
  print(f"vin: {vin}   expected: {expected_platform: <30}    fuzzy: {fuzzy_fingerprint: <30}    correct: {correct}")
  if not correct:
    print(f"  source: {surce}")
    print(f"  fuzzy: {fuzzy}")
    print("  car_fw:")
    fws = set()
    for fw in car_fw:
      if fw.brand != "ford" or fw.logging:
        continue
      fw_version = fw.fwVersion.rstrip(b"\0")
      fws.add(f"{fw.ecu} {fw_version}")
    for fw in sorted(fws):
      print(f"    {fw}")
  success += correct
  count += 1

print(f"Success rate: {success / count * 100:.2f}%")

vin: 3FTTW8E31PRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: FORD MAVERICK 1ST GEN             correct: True
vin: 3FMTK3SU0MMXXXXXX   expected: FORD MUSTANG MACH-E 1ST GEN       fuzzy: FORD MUSTANG MACH-E 1ST GEN       correct: True
vin: 3FMCR9B69NRXXXXXX   expected: FORD BRONCO SPORT 1ST GEN         fuzzy: FORD BRONCO SPORT 1ST GEN         correct: True
vin: 3FTTW8F98NRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: FORD MAVERICK 1ST GEN             correct: True
vin: WF0NXXGCHNJXXXXXX   expected: FORD FOCUS 4TH GEN                fuzzy: FORD FOCUS 4TH GEN                correct: True
vin: 3FTTW8E31PRXXXXXX   expected: FORD MAVERICK 1ST GEN             fuzzy: FORD MAVERICK 1ST GEN             correct: True
vin: WF0NXXGCHNJXXXXXX   expected: FORD FOCUS 4TH GEN                fuzzy: FORD FOCUS 4TH GEN                correct: True
vin: 3FMCR9B69NRXXXXXX   expected: FORD BRONCO SPORT 1ST GEN         fuzzy: FORD BRONCO SPORT 1ST GEN         correct: True
vin: 1FM